In [ ]:
import numpy as np
import MDAnalysis as mda
import matplotlib.pyplot as pp
import matplotlib as mpl
from matplotlib import gridspec
from MDAnalysis.analysis.rms import RMSD
from scipy import stats
from matplotlib.ticker import AutoLocator
from matplotlib.font_manager import FontProperties

%matplotlib widget

In [ ]:
textwidth = 6.50128 # latex textwidth

font0 = FontProperties()

font = font0.copy()
font.set_family('sans-serif')
font.set_style('normal')
font.set_weight('bold')
font.set_size(16)

Title = font.copy()
Title.set_size(18)

font2 = Title.copy()
font2.set_size(12)

font3 = font.copy()
font3.set_size(12)
font3.set_weight('normal')

mpl.rcParams['lines.linewidth'] = 1
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['font.size'] = 12
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['axes.labelweight'] = 'bold'
mpl.rcParams['font.family'] = 'sans-serif'

mpl.rcParams['figure.figsize'] = (textwidth, 4)

In [ ]:
colors = {
    'OPLS-AA' : '#e41a1c',
    'opt-OPLS-AA': '#4daf4a',
    'CTPOL' : '#377eb8',
    'opt-CTPOL' : '#ff7f00',
}

# Plot 1 $\times$ 40ns all force-fields

In [ ]:
psf = 'output.pdb'
ref = 'trajectories/input.pdb'

trajes = {
    'OPLS-AA' : "trajectories/MD-1ZNF/OPLS-AA/centered_output.dcd",
    'opt-OPLS-AA': "trajectories/MD-1ZNF/opt-OPLS-AA/centered_output.dcd",
    'CTPOL' : "trajectories/MD-1ZNF/CTPOL/centered_output.dcd",
    'opt-CTPOL' : "trajectories/MD-1ZNF/opt-CTPOL/centered_output.dcd",
}

TimeStep = 0.02

selections = {
    "backbone" : "backbone",
    # r"Zn-Cys$_2$His$_2$" : "name ZN or resname CYS or resname HIS",
    r"binding site" : "name ZN or name SG or (name NE2 and resname HIS) "
}

def get_density(vals):
    kernel = stats.gaussian_kde(vals)

    x = np.linspace(min(vals), max(vals))
    y = kernel(x)

    return x, y

In [ ]:
unis = {param : mda.Universe(psf, dcd) for param, dcd in trajes.items()}
refuni = mda.Universe(ref)


fig = pp.figure()

gs = gridspec.GridSpec(len(selections), 3)
gs.update(hspace=0.01, wspace=0.01)
for n, sel in enumerate(selections.items()):
    print(sel)
    if n == 0:
        ax_left = fig.add_subplot(gs[n, 0:2])
        ticks = ax_left.get_xticks
    else:
        ax_left = fig.add_subplot(gs[n, 0:2], sharex=ax_left)#, sharey=ax_left)
        
    ax_right = fig.add_subplot(gs[n, 2], sharey=ax_left)

    if n < len(selections) - 1 :
        ax_left.xaxis.set_visible(False)
    else:
        ax_left.set_xlabel(r"Time (ns)", )

    ax_right.yaxis.set_visible(False)
    ax_right.xaxis.set_visible(False)

    ax_left.text(
        0.5, n*0.72 + 0.08, sel[0], 
        horizontalalignment='left',
        verticalalignment='bottom',
        transform=ax_left.transAxes,
        
    )
    for param, uni in unis.items():
        rmsd = RMSD(uni, select=sel[1])
        rmsd.run()
        time = rmsd.results['rmsd'][:,0]*TimeStep
        ax_left.plot(
            time,
            rmsd.results['rmsd'][:,2],
            label = param,
            color = colors[param]
        )

        x, y = get_density(rmsd.results['rmsd'][:,2])
        ax_right.plot(y, x, label=param, color = colors[param])
    
    ref_rmsd = RMSD(refuni, select=sel[1])
    ref_rmsd.run()
    x, y = get_density(ref_rmsd.results['rmsd'][:,2])
    ax_right.plot(y, x, linestyle='--', color='k', label='NMR')
   
    ax_right.set_xlim(left=0)
    ax_left.set_xlim(min(time), max(time)) 

ax_right.legend()
fig.text(
    0.02, 0.5, r"RMSD $(\AA)$", 
    va='center', rotation='vertical', 
    font=font
)

fig.subplots_adjust(left=0.1, right=0.975, top=0.975, bottom=0.15)

In [ ]:
pp.savefig("Images/rmsd_density.pdf", dpi=300)


# Plot 40 $\times$ 1ns CTPOL and opt-CTPOL

In [ ]:
trajes = {
    'CTPOL' : "trajectories/40_1ns/CTPOL/centered_output.dcd",
    'opt-CTPOL' : "trajectories/40_1ns/opt-CTPOL/centered_output.dcd",
}

TimeStep = 0.02

In [ ]:
unis = {param : mda.Universe(psf, dcd) for param, dcd in trajes.items()}
refuni = mda.Universe(ref)


fig = pp.figure()

gs = gridspec.GridSpec(len(selections), 3)
gs.update(hspace=0.01, wspace=0.01)
for n, sel in enumerate(selections.items()):
    print(sel)
    if n == 0:
        ax_left = fig.add_subplot(gs[n, 0:2])
        ticks = ax_left.get_xticks
    else:
        ax_left = fig.add_subplot(gs[n, 0:2], sharex=ax_left)#, sharey=ax_left)
        
    ax_right = fig.add_subplot(gs[n, 2], sharey=ax_left)

    if n < len(selections) - 1 :
        ax_left.xaxis.set_visible(False)
    else:
        ax_left.set_xlabel(r"Time (ns)", )

    ax_right.yaxis.set_visible(False)
    ax_right.xaxis.set_visible(False)

    ax_left.text(
        0.5, n*0.72 + 0.08, sel[0], 
        horizontalalignment='left',
        verticalalignment='bottom',
        transform=ax_left.transAxes,
        
    )
    for param, uni in unis.items():
        rmsd = RMSD(uni, select=sel[1])
        rmsd.run()
        time = rmsd.results['rmsd'][:,0]*TimeStep
        ax_left.plot(
            time,
            rmsd.results['rmsd'][:,2],
            label = param,
            color = colors[param]
        )

        x, y = get_density(rmsd.results['rmsd'][:,2])
        ax_right.plot(y, x, label=param, color = colors[param])
    
    ref_rmsd = RMSD(refuni, select=sel[1])
    ref_rmsd.run()
    x, y = get_density(ref_rmsd.results['rmsd'][:,2])
    ax_right.plot(y, x, linestyle='--', color='k', label='NMR')
   
    ax_right.set_xlim(left=0)
    ax_left.set_xlim(min(time), max(time)) 

    for i in range(1, 40):
        ax_left.axvline(
            i, 
            linestyle=(0, (5,1)), 
            color='k', 
            alpha=0.5, 
            linewidth=0.5
        )
           
ax_right.legend()
fig.text(
    0.02, 0.5, r"RMSD $(\AA)$", 
    va='center', rotation='vertical', 
    font=font
)

fig.subplots_adjust(left=0.1, right=0.975, top=0.975, bottom=0.15)

In [ ]:
fig.savefig('Images/rmsd_density_40x1.pdf', dpi=300)